# Freedmen's Bureau Analysis 

## Task #0: Configure an LLM

### Import libraries that we'll use in this notebook

In [1]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import json
import pandas as pd
import ast
import textwrap


## Task #1: Parse Entitities from Simple Examples (Warmup)

### Samples to be able to parse correctly

In [2]:

inputs = [
    "John James agrees to pay $50/month to RJ Hampshire for work on the Farm",
    "Elizabeth James will pay $30 per month to Levi Rodgers for Gardening",
    "Johnson Ollaman will pay $1.25 per day to both John Smith and Jane Smith for teaching the children of the community",
    "Claire Daniels charges $50 weekly to local community members for cooking classes, emphasizing the joy of healthy eating.",
    "Marcus Wellby commits to donating $500 annually to the Green Earth Foundation for environmental conservation efforts.",
    "Dr. Helena Russell charges $100 per hour for providing guidance and support to medical students, aiming to enhance their clinical skills and knowledge.",
    "Keith Galli charges $0 to watch his YouTube content; the least you could do is smash that like button and subscribe, hehehe xD",
    "The local sports club agrees to pay $75 each to coaches Sarah Miller, Danny Glover, Alex Reed, and Jamie Fox for conducting a weekend sports clinic.",
    """This Agreement made this 14th day of August A.D. 1865, by and between F.R.J. Terry of the county of Copiah and State of Mississippi of the first part, and the person hereinafter named and undersigned, Freedmen of the second part [[?]] That for the purpose of working in the [[?]] known as Beagley's [[?]] Yard in the county aforesaid for two months commencing on the 14th day of August 1865 and terminating on the 14th day of October 1865. The said F.R.J. Terry party of the first part, in consideration of the [[?]] and conditions hereinafter mentioned on the part of the party of the second part agrees to pay said laborer "10" ten dollars per month and furnish free of charge clothing and good of good quality and sufficient quantity, good and sufficient quarters, and kind and humane treatment. And it is further agreed that in case the said F.R.J. Terry shall fail, neglect, or refuse to fulfill any of the obligations assumed by him, he shall besides the legal recourse left to the party aggrieved render this contract liable to amendment by the Provost Marshal of Freedmen. And it is agreed on the part of the party of second part that he will well and faithfully perform such labor as the said F.R.J. Terry may require of him for the time aforesaid, nor exceeding ten hours per day in summer and nine hours in winter. And in case the said laborer shall absent himself from or refuse to perform the labor herein promised, he shall loose the time and be punished as such manner as the Provost Marshal shall deem propper.""",
]

outputs = [
    [{"payer": "John James", "recipient": "RJ Hampshire", "amount": 50, "pay frequency": "monthly", "description": "farming"}],
    [{"payer": "Elizabeth James", "recipient": "Levi Rodgers", "amount": 30, "pay frequency": "monthly", "description": "gardening"}],
    [{"payer": "Johnson Ollaman", "recipient": "John Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}, {"payer": "Johnson Ollaman", "recipient": "Jane Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}],
    [{"payer": "Claire Daniels", "recipient": "Local community members", "amount": 50, "pay frequency": "weekly", "description": "cooking classes"}],
    [{"payer": "Marcus Wellby", "recipient": "Green Earth Foundation", "amount": 500, "pay frequency": "yearly", "description": "donation for environmental conservation"}],
    [{"payer": "Dr. Helena Russell", "recipient": "Medical students", "amount": 100, "pay frequency": "hourly", "description": "mentorship and clinical skill enhancement"}],
    [{"payer": None, "recipient": "Keith Galli", "amount": 0, "pay frequency": None, "description": "YouTube content"}],
    [{"payer": "The local sports club", "recipient": "Sarah Miller", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Danny Glover", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Alex Reed", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Jamie Fox", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"}],
    [{"payer": "F.R.J. Terry", "payee": "Freedmen", "amount":10, "pay frequency": "monthly", "description": "working in the yard"}],
]

### Your code here

In [3]:
system_message = textwrap.dedent("""
  From the following historical document text, please grab out the following items and return with the following format JSON
  {{ "results":
  [{{
    "payer": "<Payer Name>",
    "recipient": "<Payee Name>",
    "amount": <Amount Paid>, # This is the amount of money in USD
    "pay frequency": "<Frequency Paid>" # Only options are "hourly, "daily", "monthly", "weekly", "yearly", "one-time", or "other"
    "description": "<Description of Work>" # This is the description of the work being done
  }},
  {{<ITEM 2>}}, {{Item 3>}}, ...]
  }}
  So for the following text:
  "Jane Doe will pay $50/month to John Smith for work on the Farm"

  the output text generated would be:
  {{ "results":
  [{{
    "payer": "Jane Doe",
    "recipient": "John Smith",
    "amount": 50,
    "pay frequency": "monthly"
    "description": "work on the farm"
  }}]
  }}

  IMPORTANT: Return only the dictionary mapping results to a list of Python dictionary objects and nothing more. It is possible there are no matches, in which case return results field being an empty list.
                                 
  If there are multiple matches, list the full JSON details for each as an item in the results array
  """
)

In [4]:
# chat = ChatOpenAI(model_name="gpt-4-1106-preview", response_format={"type":"json_object"})
chat = ChatOpenAI()
# chat = ChatOllama(model="llama2", format="json")

def get_output(input):
    messages = [SystemMessage(content=system_message), HumanMessage(content=inputs[0]), AIMessage(content=str(outputs[0])), HumanMessage(content=input)]
    output = chat.invoke(messages)
    return output.content

def parse_output(output):
    print(output)
    return json.loads(output)

print(inputs[1])

test = get_output(inputs[1])
answer = parse_output(test)

answer["results"]





Elizabeth James will pay $30 per month to Levi Rodgers for Gardening
{
"results": [
{
"payer": "Elizabeth James",
"recipient": "Levi Rodgers",
"amount": 30,
"pay frequency": "monthly",
"description": "Gardening"
}
]
}


[{'payer': 'Elizabeth James',
  'recipient': 'Levi Rodgers',
  'amount': 30,
  'pay frequency': 'monthly',
  'description': 'Gardening'}]

## Task #2a: Grab Labor Contract Rows from Kaggle

In [5]:
import pandas as pd

df = pd.read_csv('./input/contract-records.csv')
df['transcription_text'] = df['transcription_text'].str.replace('_x000D_',' ')

aa = df[df['sub_category'] == 'Apprenticeship Agreement']
aa.head()

aa.to_csv('./input/apprenticeship-agreements-test.csv', index=False)
# apprenticeships = df[df['sub_category']=='Labor Contracts']
# labor.head()

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# # pd.reset_option('display.max_colwidth')
# # pd.reset_option('display.max_columns')
# # pd.reset_option('display.max_rows')



# labor.head()


## Task #2: Connect pages that belong to the same document

In [6]:
aa = pd.read_csv('./input/apprenticeship-sample.csv')
aa.head()

,project_id,category,sub_category,transcription_text,document_url,expected_id
0,15884,Contracts,Apprenticeship Agreement,Apprenticeship \n \nJames Conoly \n \nMary 12 ...,https://transcription.si.edu/transcribe/15884/...,0
1,15884,Contracts,Apprenticeship Agreement,Indentures of Apprenticeship. \nState of North...,https://transcription.si.edu/transcribe/15884/...,0
2,15884,Contracts,Apprenticeship Agreement,Indenture of Apprenticeship \n \nSept 14th 186...,https://transcription.si.edu/transcribe/15884/...,1
3,15884,Contracts,Apprenticeship Agreement,Indenture of Apprenticeship \nState of North C...,https://transcription.si.edu/transcribe/15884/...,1
4,15884,Contracts,Apprenticeship Agreement,Indenture of Apprenticeship \n \nSept 14th 186...,https://transcription.si.edu/transcribe/15884/...,2


In [37]:
import ast
import time

def from_same_document(document1, document2):
    system_message = """
    Your job is to determine whether the following two documents are from the same document or not. Please return a boolean value of True or False. 
    One of the key things to look out for is the same names, locations, or dates referred to in both documents.

    Document 1:
    {document1}

    Document 2:
    {document2}

    ---
    IMPORTANT: Make sure to only return the boolean value and nothing else
    """
    
    # Fail fast if the documents are not from the same project
    if document1['project_id'] != document2['project_id']:
        return False
    
    prompt = PromptTemplate(input_variables=['document1', 'document2'], template=system_message)
    full_prompt = prompt.format(document1=document1['transcription_text'], document2=document2['transcription_text'])

    max_retries = 3  # Maximum number of retries
    retry_delay = 5  # Delay between retries in seconds

    for attempt in range(max_retries):
        try:
            response_raw = chat.invoke(full_prompt, timeout=20).content  # Assuming this is your method to call the API
            # Assuming the API response is a string that can be evaluated as a Python literal (e.g., "True" or "False")
            print("Response", response_raw.strip())
            return ast.literal_eval(response_raw.strip())
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing response: {e}")
            return False  # Or handle the error as appropriate for your application
        except Exception as e:
            print(f"An error occurred: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print("Maximum retries reached. Handling as a failure.")
                return False  # Or handle as appropriate

In [38]:
import time

def group_documents_in_df(df):
    merge_id = 0
    for i in range(0, len(df)):
        if i%20 == 0:
            print(f"Processing {i} of {len(df)}")
            time.sleep(5)
        # Access the row index instead of the row itself

        index1 = df.index[i]
        index2 = df.index[i+1] if i + 1 < len(df) else None

        # Use loc to access the rows
        document1 = df.loc[index1]
        document2 = df.loc[index2] if index2 is not None else None

        if document2 is not None:
            if from_same_document(document1, document2):
                # Set the value using loc
                df.loc[index1, 'merge_id'] = int(merge_id)
                df.loc[index2, 'merge_id'] = int(merge_id)
                continue
            
        df.loc[index1, 'merge_id'] = int(merge_id)
        merge_id += 1
    
    return df


In [39]:
aa_full = pd.read_csv('./input/apprenticeship-agreements.csv')

In [40]:
group_documents_in_df(aa_full)

Processing 0 of 2405
Response True
Response True
Response True
Response True
Response False
Response True
Response False
Response False
Response False
Response False
Response False
Response False
Response False
Response False
Response False
Processing 20 of 2405
Response False
Response False
Response True
Response False
Response True
Response False
Response True
Response False
Response True
Response True
Response True
Response True
Response False
Response True
Response True
Response True
Response True
Response False
Response True
Processing 40 of 2405
Response True
Response True
Response False
Response False
Response True
Response False
Response True
Response False
Response True
Response True
Response True
Response False
Response True
Response False
Response True
Response False
Response True
Response False
Response True
Response True
Processing 60 of 2405
Response True
Response False
Response True
Response False
Response True
Response False
Response True
Response False
Response True
Re

,project_id,category,sub_category,transcription_text,document_url,merge_id
0,11406,Contracts,Apprenticeship Agreement,Copy \n \nAgreement of Apprenticeship \nBu \n...,https://transcription.si.edu/transcribe/11406/...,0.0
1,11406,Contracts,Apprenticeship Agreement,[H 132 ENCL] \n \nCopy \n \nAgreement of Appr...,https://transcription.si.edu/transcribe/11406/...,0.0
2,11406,Contracts,Apprenticeship Agreement,is agreed on the part of the party of the firs...,https://transcription.si.edu/transcribe/11406/...,0.0
3,11406,Contracts,Apprenticeship Agreement,continuance of this agreement & finally that t...,https://transcription.si.edu/transcribe/11406/...,0.0
4,15369,Contracts,Apprenticeship Agreement,"[[preprinted]] \nBureau of Refugees, Freedman ...",https://transcription.si.edu/transcribe/15369/...,1.0
...,...,...,...,...,...,...
2400,41539,Contracts,Apprenticeship Agreement,State of Tennessee } \nShelby County. } \n...,https://transcription.si.edu/transcribe/41539/...,1055.0
2401,41539,Contracts,Apprenticeship Agreement,It is further agreed by the parties to this co...,https://transcription.si.edu/transcribe/41539/...,1055.0
2402,41539,Contracts,Apprenticeship Agreement,State of Tennessee \nShelby County. \n \nThis ...,https://transcription.si.edu/transcribe/41539/...,1056.0
2403,41539,Contracts,Apprenticeship Agreement,It is further Agreed to by parties of this con...,https://transcription.si.edu/transcribe/41539/...,1056.0


In [41]:
aa_full.to_csv('./input/updated_sheet_new.csv', index=False)

In [43]:

aa_full.head()

# Set the display option to show full column value
pd.set_option('display.max_colwidth', None)

# Call aa.head() to display the dataframe with full column value
aa_full.head()


,project_id,category,sub_category,transcription_text,document_url,merge_id
0,11406,Contracts,Apprenticeship Agreement,Copy \n \nAgreement of Apprenticeship \nBu \nMrs. Kate V.Chamblin \nof the first part and \n \n1st Lt. Geo. W. Rollins \nV.R.C. U.S. Army Agt \nfor Carroll Parish La. of \nthe second part \n \nMinor Orphans Freed \nJames Samenett \nBetty Taylor \nMary Taylor \n \nLake Providence \nCarroll Parish La \nDated Oct. 1 1866 \nExec'd Nov. 3 1866 \n,https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00439,0.0
1,11406,Contracts,Apprenticeship Agreement,"[H 132 ENCL] \n \nCopy \n \nAgreement of Apprenticeship. \n \nThis agreement in two parts made & entered into this 1"" day of October A. D. 1866 by & between Mrs Kate V. Chambliss of the first part and 1st Lieut Geo. W. Rollins Vet Res. Corps U.S.A. Agent for Carroll Parish La. Bur of Ref. Freedn & Aband Lands La and by virtue of authority contained in Circular No 25 dated Hd. Qrs. Bur Ref Freedn & Abd Lands New Orleans Louisiana Octr 31"" 1865 Guardian for Minors & orphans of Freedmen for Carroll Parish La party hereto of the second part. Witnesseth That James Samenett, Betty Taylor and Mary Taylor minor orphans of African decent are hereby bound & apprenticed to service to the said Mrs Kate V. Chambliss party of the first part & undersigned during their years of minority commencing & ending as follows: James Samenett aged 14 years, commencing on the 1"" day of Oct. A. D. 1866 & terminating on the 1"" day of October 1870. Betty Taylor aged 9 years commencing on the 1"" day of Oct. A. D. 1866 & ending on the 1"" day of Oct. A. D. 1872. Mary Taylor aged 7 years commencing on the 1"" day of October A. D. 1866 & terminating on the 1"" day of October A. D. 1874. And it",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00440,0.0
2,11406,Contracts,Apprenticeship Agreement,"is agreed on the part of the party of the first part & undersigned Mrs Kate V. Chambliss that for the consideration of the faithfull services to be rendered by the within named & said minor orphans the said minor orphans shall receive comfortable clothing, board medical treatment when sick, a reasonable amount of schooling and permission to attend church each Sabbath and at the end or termination of their term of apprenticeship the said minor orphans shall be allowed to retain all articles of their personal apparel. \n \nThe said parties do hereby mutually agree that all laws or parts of laws enacted or that may be enacted by the United States or the State of Louisiana establishing laws for the welfare or government of Freedmens minors or orphans or for the government of the same under laws establishing the Bur of Ref Freed & Ab^d Lands or in any way affecting the provisions of this agreement is & shall be made a part of this agreement and that all laws applying to white minors or orphans shall be extended to the said & within named minor orphans during the conti-",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00441,0.0
3,11406,Contracts,Apprenticeship Agreement,"continuance of this agreement & finally that this agreement shall expire & terminate when the aforesaid minor male orphan shall have arrived at the age of eighteen years and the minor female orphans shall have arrived at the age of fifteen years respectively. \n \nIn testimony whereof the said parties have hereunto affixed their names to this agreement. Done at Lake Providence Louisiana Parish of Carroll on the third day of November A. D. 1866. \n \nsigned Kate V. Chambliss \nsigned Geo. W Rollins \n1st Lieut. VRC. U.S. Army \nAgent for Carrol Parish \nLa. Bureau of R. F. & A Lands \nLouisiana \n \nExecuted in Presence of \nSigned Ben C. Johnson \nJackson Chambliss \nJohn A. Ginst [[?]] \n \n[[image - three boxes oriented vertically representing Internal Revenue Stamps that appeared on original document, reading as follows: 2¢ / Int Rev / [[illegible]]; a single illegible initial appears beneath each ""stam

In [46]:
df = pd.read_csv('./input/apprenticeship-agreements-updated.csv')
# Step 1 & 2: Group by 'merge_id' and concatenate 'transcription_text'
aggregated_text = df.groupby('merge_id')['transcription_text'].agg('\n'.join).reset_index()

# Step 3: Drop duplicate 'merge_id' rows, keeping the first occurrence
df = df.drop_duplicates(subset='merge_id')

# Step 4: Merge the aggregated text back into the original DataFrame
df = pd.merge(df, aggregated_text, on='merge_id', how='left', suffixes=('', '_aggregated'))

# Now, 'transcription_text_aggregated' contains the concatenated text
# You might want to rename or drop the original 'transcription_text' column as needed
df.drop(columns=['transcription_text'], inplace=True)

In [47]:
df.to_csv('./input/merged_apprenticeship_agreements.csv', index=False)

## Task #3: Grab Apprenticeship info from CSV

In [82]:
df = pd.read_csv('./input/merged_apprenticeship_agreements.csv')

In [83]:
system_message = textwrap.dedent("""
  From the following historical document text, please grab out the following items and return with the following format JSON
  {{ "results":
  [{{
    "official": "<Freedmen Bureau / Gov Official / Military Official Name>",
    "mentor": "< Person agreeing to take in apprentice >",
    "apprentice_name": "<Name of apprentice>", # The full name (if known) of the apprentice, use first name if all that is known
    "apprentice_age": <Age of the apprentice>, # int value (years), None if not stated
    "state": "< State contract is made >", # If not stated, use None
    "county": "< County contract is made >" # If not stated, use None
  }},
  {{<ITEM 2>}}, {{Item 3>}}, ...]
  }}
  So for the following text:
  "Roger Jones agrees to take in apprentice John Smith, aged 14, in the state of Mississippi, in the county of Copiah with witness John Doe"

  the output text generated would be:
  {{ "results":
  [{{
    "official": "John Doe",
    "mentor": "Roger Jones",
    "apprentice_name": "John Smith",
    "apprentice_age": 14, 
    "state": "Mississippi",
    "county": "Copiah"
  }}]
  }}

  IMPORTANT: Return only the dictionary mapping results to a list of Python dictionary objects and nothing more. It is possible there are no matches, in which case return results field being an empty list.
                                 
  If there are multiple matches, list the full JSON details for each as an item in the results array
  """
)

In [84]:
chat = ChatOpenAI()
# chat = ChatOllama(model="llama2", format="json")

def get_output(input):
    try:
        messages = [SystemMessage(content=system_message), HumanMessage(content=input)]
        output = chat.invoke(messages, timeout=20)
        return output.content
    except Exception as e:
        print(e)
        return None

def parse_output(output):
    print(output)
    return json.loads(output)

inputs = df['transcription_text_aggregated'].tolist()
print(inputs[0])

test = get_output(inputs[0])
answer = parse_output(test)

answer["results"]


Copy 
 
Agreement of Apprenticeship 
Bu  
Mrs. Kate V.Chamblin  
of the first part and 
 
1st Lt. Geo. W. Rollins 
V.R.C. U.S. Army Agt 
for Carroll Parish La. of  
the second part 
 
Minor Orphans Freed 
James Samenett 
Betty Taylor 
Mary Taylor    
 
Lake Providence 
Carroll Parish La 
Dated Oct. 1 1866 
Exec'd Nov. 3 1866 

[H 132 ENCL]  
 
Copy 
 
Agreement of Apprenticeship. 
 
This agreement in two parts made & entered into this 1" day of October A. D. 1866 by & between Mrs Kate V. Chambliss of the first part and 1st Lieut Geo. W. Rollins Vet Res. Corps U.S.A. Agent for Carroll Parish La. Bur of Ref. Freedn & Aband Lands La and by virtue of authority contained in Circular No 25 dated Hd. Qrs. Bur Ref Freedn & Abd Lands New Orleans Louisiana Octr 31" 1865 Guardian for Minors & orphans of Freedmen for Carroll Parish La party hereto of the second part. Witnesseth That James Samenett, Betty Taylor and Mary Taylor minor orphans of African decent are hereby bound & apprenticed to servi

[{'official': 'Ben C. Johnson',
  'mentor': 'Kate V. Chambliss',
  'apprentice_name': 'James Samenett',
  'apprentice_age': 14,
  'state': 'Louisiana',
  'county': 'Carroll Parish'},
 {'official': 'Ben C. Johnson',
  'mentor': 'Kate V. Chambliss',
  'apprentice_name': 'Betty Taylor',
  'apprentice_age': 9,
  'state': 'Louisiana',
  'county': 'Carroll Parish'},
 {'official': 'Ben C. Johnson',
  'mentor': 'Kate V. Chambliss',
  'apprentice_name': 'Mary Taylor',
  'apprentice_age': 7,
  'state': 'Louisiana',
  'county': 'Carroll Parish'}]

In [85]:
## Iterate over the entire df and add a column saving the json results
df['output_json'] = None
for index, row in df.iterrows():
    if index > 0 and index % 10 == 0:
        print(f"Processing {index} of {len(df)}")
        time.sleep(10)

    if row['output_json'] is None:
        text = row['transcription_text_aggregated']
        output = get_output(text)
        df.loc[index, 'output_json'] = output
        print(index)

0
1
2
3
4
5
6
7
8
9
Processing 10 of 1058
Request timed out.
10
11
12
13
14
15
16
17
18
19
Processing 20 of 1058
20
21
22
23
24
25
26
27
28
29
Processing 30 of 1058
30
31
32
33
34
35
36
37
38
39
Processing 40 of 1058
40
41
42
43
44
45
46
47
48
49
Processing 50 of 1058
50
51
52
53
54
55
56
57
58
59
Processing 60 of 1058
60
61
62
63
64
65
66
67
68
69
Processing 70 of 1058
70
71
72
73
74
75
76
77
78
79
Processing 80 of 1058
80
81
82
83
84
85
86
87
88
89
Processing 90 of 1058
90
91
92
93
94
95
96
97
98
99
Processing 100 of 1058
100
101
102
103
104
105
106
107
108
109
Processing 110 of 1058
110
111
112
113
114
115
116
117
118
119
Processing 120 of 1058
120
121
122
123
124
125
126
127
128
129
Processing 130 of 1058
130
131
132
133
134
135
136
137
138
139
Processing 140 of 1058
140
141
142
143
144
145
146
147
148
149
Processing 150 of 1058
150
151
152
153
154
155
156
157
158
159
Processing 160 of 1058
160
161
162
163
164
165
166
167
168
169
Processing 170 of 1058
170
171
172
173
174
175
176
1

In [86]:
df.to_csv('./input/outputs.csv', index=False)

In [87]:
outputs = pd.read_csv('./input/outputs.csv')

In [88]:
output_list = []
for index, row in outputs.iterrows():
    if row['output_json'] is not None:
        try:
            output = json.loads(row['output_json'])
            output_list.append(output['results'])
        except Exception as e:
            print(e)
            print(row['output_json'])
    

Expecting value: line 7 column 19 (char 140)
{
"results": [
{
"official": "Lt. A. G. Tennant",
"mentor": "Dr. W. A. B. Norcum",
"apprentice_name": "Mark Satterfield",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": "Lt. A. G. Tennant",
"mentor": "William Coffield",
"apprentice_name": "John",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": "David A. Holly",
"mentor": "David A. Holly",
"apprentice_name": "Andrew",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": None,
"mentor": "Thomas W. Hudgins",
"apprentice_name": "Flora Hudgins",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": None,
"mentor": "Thomas C. Spruell",
"apprentice_name": "Charles Wynn",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": None,
"mentor": "Susan M. Bunch",
"apprentice_name": "Reuben Bunch",
"apprentice_age": None,
"state": None,
"county": None
},
{
"official": None,
"mentor": "W. H. Leary",
"apprenti

In [109]:
counts = {}
field = "mentor"
for output in output_list:
    for item in output:
        if item.get(field) not in counts:
            counts[item.get(field)] = 1
        else:
            counts[item.get(field)] += 1
    

In [111]:
print(counts)
counts_copy = counts.copy()
for key, value in counts_copy.items():
    if isinstance(value,int) and value < 8:
        counts.pop(key)

counts.pop(None)

import matplotlib.pyplot as plt

# Assuming 'counts' is the dictionary of values mapped to their counts
values = list(counts.keys())
frequency = list(counts.values())

plt.bar(values, frequency)
plt.xlabel('Values')
plt.ylabel('Counts')
plt.title('Histogram')
plt.show()


{'James Conoly': 11, 'Clinton A. Cilley': 18, 'J H Bassler': 23, 'Thos. J. Richards': 8, 'Asa Teal': 21, 'H H Foster': 29, 'John Barden': 8, 'Henry W. Harrington': 9}


KeyError: None